Preaparing virtual enviroment and getting data 

In [ ]:
# Preparing virtual enviroment
!pip install --upgrade nibabel
!rm public.zip
!pip install --upgrade nibabel
!pip install segmentation-models

# get pictures and unpacked them
!wget "https://putpoznanpl-my.sharepoint.com/:u:/g/personal/dominik_pieczynski_put_poznan_pl/EWIZ_xm8wXpMjQDgF2VQ1csB4QuHPKoj5vDpj6CQi9p-AA?e=yQr6fn&download=1" -O public.zip
!unzip -q public.zip

Importing all necessary packages 

In [ ]:
import numpy as np
import nibabel as nib
from pathlib import Path 
import matplotlib.pyplot as plt
import os
import cv2 as cv
from typing import Tuple, List

from sklearn.model_selection import train_test_split
import tensorflow as tf
from segmentation_models.losses import DiceLoss
from tensorflow.keras.models import load_model
from segmentation_models.metrics import IOUScore
from segmentation_models import Unet
import scipy
import keras

from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from google.colab import drive



Using TensorFlow backend.


Segmentation Models: using `keras` framework.


All functions used in project

In [ ]:
# functions provided by tutor used for loading, saving and visualizing raw data

def load_raw_volume(path: Path) -> Tuple[np.ndarray, np.ndarray]:
  data: nib.Nifti1Image = nib.load(str(path))
  data = nib.as_closest_canonical(data)
  raw_data = data.get_fdata(caching='unchanged', dtype=np.float32)
  return raw_data, data.affine

def load_labels_volume(path: Path) -> np.ndarray:
  return load_raw_volume(path)[0].astype(np.uint8)


def save_labels(data: np.ndarray, affine: np.ndarray, path: Path):
  nib.save(nib.Nifti1Image(data, affine), str(path))


def show_slices(slices: List[np.ndarray]):
   fig, axes = plt.subplots(1, len(slices))
   for i, data_slice in enumerate(slices):
       axes[i].imshow(data_slice.T, cmap="gray", origin="lower")

# img operations functions

def img_normalize(img):
  img = img/np.amax(img)*255
  return img

def get_slice_from_raw_data(raw_data):
  return [raw_data[raw_data.shape[0] // 2], raw_data[:, raw_data.shape[1] // 2], raw_data[:, :, raw_data.shape[2] // 2]]

def img_padding(img, size):
  extended_img = np.zeros(size, dtype=np.uint8)
  extended_img[:img.shape[0], :img.shape[1]] = img
  return extended_img

def img_unpadding(img, size):
  return img[:size[0], :size[1]]

# drive operations related functions

def cerate_storage_folders(paths):
  for path in paths:
    if not os.path.exists(path):
      os.mkdir(path)

# saving png to drive functions

def save_raw_img_to_drive(slices, saving_path, folders_names, file_name):
  
  for i in range(3):
    current_slice_path = Path(saving_path, folders_names[i], file_name + '.png' )
    cv.imwrite(str(current_slice_path), img_normalize(slices[i]), [cv.IMWRITE_PNG_COMPRESSION, 0]) 

def save_mask_to_drive(slices, saving_path, folders_names, file_name):
  
  for i in range(3):
    current_slice_path = Path(saving_path, folders_names[i], file_name + '.png' )
    cv.imwrite(str(current_slice_path), slices[i], [cv.IMWRITE_PNG_COMPRESSION, 0])

# dataset operations related functions

def get_file_names_from_path(dataset_path):
  file_names = []
  for scan_path in dataset_path.iterdir():
    if not scan_path.name.endswith('mask.nii.gz'):
      file_name = os.path.splitext(scan_path.name)
      file_name = os.path.splitext(file_name[0])
      file_names.append(file_name[0])
  return file_names

def get_directory_names_from_path(dataset_paths):
  directory_names = []
  for directory_path in dataset_paths.iterdir():
    directory_names.append(directory_path.name)
  return directory_names

def train_valid_split(file_names):
  train_file_names, valid_file_names, temp1, temp2 =  train_test_split(file_names, file_names, test_size = valid_set_percentage, random_state = random_state, shuffle = True)
  return train_file_names, valid_file_names

def pad_all_data(path, size, datasets_folders_names):
  for directory_path in path.iterdir():
    if directory_path.name in datasets_folders_names:
      for directory_name in directory_path.iterdir():
        for img_path in directory_name.iterdir():

          try:
              img = cv.imread(str(img_path), 0)
          except Exception as e:
              continue
          cv.imwrite(str(img_path), img_padding(img, size))

# predictions functions

def get_prediction(data_slice, model, padding_shape):
  data_slice = img_normalize(data_slice)
  size = data_slice.shape[:2]
  data_slice = img_padding(data_slice, padding_shape)
  predict_result = model.predict(data_slice[None, :])
  predict_result = predict_result.squeeze()
  predict_result[predict_result > 0.5] = 1
  predict_result[predict_result != 1] = 0
  return img_unpadding(predict_result, size)




Variables used in project

In [ ]:
# paths variables

main_path = '/content/drive/My Drive/zpo_project'

main_folders_names = ['data', 'models', 'results']
datasets_folders_names = ['train', 'valid']
scans_folders_names = ['imgs_side','imgs_front','imgs_top']
masks_folders_names = ['masks_side','masks_front','masks_top']
predict_folders_names = ['result_1', 'result_2']

first_dataset_path = Path('/content/FirstDataset/')
second_dataset_path = Path('/content/SecondDataset/')

# used in first dataset
mask_ext = '_mask.nii.gz'
scan_ext = '.nii.gz'

# used in second dataset
mask_file_name = 'mask.nii.gz'
scan_file_name = 'T1w.nii.gz'

valid_set_percentage = 0.15
random_state = 42

padding_shape = (512, 512)

input_shape = (512, 512, 1)

loss = DiceLoss()
metric = IOUScore()


Prepare drive 

In [ ]:
# mount drive 
drive.mount('/content/drive')

# create necessary folders
paths = ['/content/drive/My Drive/zpo_project',
         '/content/drive/My Drive/zpo_project/data',
         '/content/drive/My Drive/zpo_project/data/train',
         '/content/drive/My Drive/zpo_project/data/train/imgs_side',
         '/content/drive/My Drive/zpo_project/data/train/imgs_front',
         '/content/drive/My Drive/zpo_project/data/train/imgs_top',
         '/content/drive/My Drive/zpo_project/data/train/masks_side',
         '/content/drive/My Drive/zpo_project/data/train/masks_front',
         '/content/drive/My Drive/zpo_project/data/train/masks_top',
         '/content/drive/My Drive/zpo_project/data/valid',
         '/content/drive/My Drive/zpo_project/data/valid/imgs_side',
         '/content/drive/My Drive/zpo_project/data/valid/imgs_front',
         '/content/drive/My Drive/zpo_project/data/valid/imgs_top',
         '/content/drive/My Drive/zpo_project/data/valid/masks_side',
         '/content/drive/My Drive/zpo_project/data/valid/masks_front',
         '/content/drive/My Drive/zpo_project/data/valid/masks_top',
         '/content/drive/My Drive/zpo_project/models',
         '/content/drive/My Drive/zpo_project/results',
         '/content/drive/My Drive/zpo_project/results/',
         '/content/drive/My Drive/zpo_project/results/result_1',
         '/content/drive/My Drive/zpo_project/results/result_2'
         ]
cerate_storage_folders(paths)



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Preparing raw data for training (padding and saving as png on drive)

In [ ]:
train_save_path = Path('/content/drive/My Drive/zpo_project/data/train')
valid_save_path = Path('/content/drive/My Drive/zpo_project/data/valid')

# first dataset 

train_path = os.path.join(first_dataset_path, 'train')
file_names = get_file_names_from_path(Path(train_path))
 
train_file_names, valid_file_names = train_valid_split(file_names)

for file_name in train_file_names:

  raw_volume = None
  mask_volume = None
  mask_path = os.path.join(train_path, file_name+mask_ext)
  scan_path = os.path.join(train_path, file_name+scan_ext)
 
  # load mask and scan file, check for errors
  try:
    mask_volume = load_labels_volume(mask_path)
    raw_volume, affine = load_raw_volume(scan_path)
  except:
    continue
  
  # if loaded, get images into list
  scans = get_slice_from_raw_data(raw_volume)
  masks = get_slice_from_raw_data(mask_volume)
 
  save_raw_img_to_drive(scans, train_save_path, scans_folders_names, file_name)
  save_mask_to_drive(masks, train_save_path, masks_folders_names, file_name)

for file_name in valid_file_names:
 
  raw_volume = None
  mask_volume = None
  mask_path = os.path.join(train_path, file_name+mask_ext)
  scan_path = os.path.join(train_path, file_name+scan_ext)
 
  # load mask and scan file, check for errors
  try:
    mask_volume = load_labels_volume(mask_path)
    raw_volume, affine = load_raw_volume(scan_path)
  except:
    continue
  
  # if loaded, get images into list
  scans = get_slice_from_raw_data(raw_volume)
  masks = get_slice_from_raw_data(mask_volume)
 
  save_raw_img_to_drive(scans, valid_save_path, scans_folders_names, file_name)
  save_mask_to_drive(masks, valid_save_path, masks_folders_names, file_name)

# second dataset 
train_path = os.path.join(second_dataset_path, 'train')
dirs_names = get_directory_names_from_path(Path(train_path))

train_dirs_names, valid_dirs_names = train_valid_split(dirs_names)

for dir_name in train_dirs_names:

  raw_volume = None
  mask_volume = None
  mask_path = os.path.join(train_path, dir_name, mask_file_name)
  scan_path = os.path.join(train_path, dir_name, scan_file_name)

   # load mask and scan file
  try:
    mask_volume = load_labels_volume(mask_path)
    raw_volume, affine = load_raw_volume(scan_path)
  except:
    continue
  
  scans = get_slice_from_raw_data(raw_volume)
  masks = get_slice_from_raw_data(mask_volume)
 
  save_raw_img_to_drive(scans, train_save_path, scans_folders_names, dir_name)
  save_mask_to_drive(masks, train_save_path, masks_folders_names, dir_name)

for dir_name in valid_dirs_names:

  raw_volume = None
  mask_volume = None
  mask_path = os.path.join(train_path, dir_name, mask_file_name)
  scan_path = os.path.join(train_path, dir_name, scan_file_name)

   # load mask and scan file
  try:
    mask_volume = load_labels_volume(mask_path)
    raw_volume, affine = load_raw_volume(scan_path)
  except:
    continue
  
  scans = get_slice_from_raw_data(raw_volume)
  masks = get_slice_from_raw_data(mask_volume)
 
  save_raw_img_to_drive(scans, valid_save_path, scans_folders_names, dir_name)
  save_mask_to_drive(masks, valid_save_path, masks_folders_names, dir_name)

Applay padding to make all images same size

In [ ]:
# padding
data_path = Path('/content/drive/My Drive/zpo_project/data')
pad_all_data(data_path, padding_shape, datasets_folders_names)

Deep learning section

Prepare virtual machine 



In [ ]:
# loading training data to colab virtual machine

# Train dataset
!mkdir /content/train
!mkdir /content/valid
!mkdir /content/test

!mkdir /content/train/img_front
!mkdir /content/train/mask_front
!mkdir /content/train/img_front/all_img
!mkdir /content/train/mask_front/all_mask

!mkdir /content/train/img_side
!mkdir /content/train/mask_side
!mkdir /content/train/img_side/all_img
!mkdir /content/train/mask_side/all_mask

!mkdir /content/train/img_top
!mkdir /content/train/mask_top
!mkdir /content/train/img_top/all_img
!mkdir /content/train/mask_top/all_mask

!mkdir /content/valid/img_front
!mkdir /content/valid/mask_front
!mkdir /content/valid/img_front/all_img
!mkdir /content/valid/mask_front/all_mask

!mkdir /content/valid/img_side
!mkdir /content/valid/mask_side
!mkdir /content/valid/img_side/all_img
!mkdir /content/valid/mask_side/all_mask

!mkdir /content/valid/img_top
!mkdir /content/valid/mask_top
!mkdir /content/valid/img_top/all_img
!mkdir /content/valid/mask_top/all_mask

!mkdir /content/test/img_front
!mkdir /content/test/img_front/all_img

!mkdir /content/test/img_side
!mkdir /content/test/img_side/all_img

!mkdir /content/test/img_top
!mkdir /content/test/img_top/all_img

# Copying to virtual machine
!cp /content/drive/My\ Drive/zpo_project/data/train/imgs_front/*.png /content/train/img_front/all_img 
!cp /content/drive/My\ Drive/zpo_project/data/train/masks_front/*.png /content/train/mask_front/all_mask

!cp /content/drive/My\ Drive/zpo_project/data/valid/imgs_front/*.png /content/valid/img_front/all_img 
!cp /content/drive/My\ Drive/zpo_project/data/valid/masks_front/*.png /content/valid/mask_front/all_mask

!cp /content/drive/My\ Drive/zpo_project/data/train/imgs_side/*.png /content/train/img_side/all_img 
!cp /content/drive/My\ Drive/zpo_project/data/train/masks_side/*.png /content/train/mask_side/all_mask

!cp /content/drive/My\ Drive/zpo_project/data/valid/imgs_side/*.png /content/valid/img_side/all_img 
!cp /content/drive/My\ Drive/zpo_project/data/valid/masks_side/*.png /content/valid/mask_side/all_mask

!cp /content/drive/My\ Drive/zpo_project/data/train/imgs_top/*.png /content/train/img_top/all_img 
!cp /content/drive/My\ Drive/zpo_project/data/train/masks_top/*.png /content/train/mask_top/all_mask

!cp /content/drive/My\ Drive/zpo_project/data/valid/imgs_top/*.png /content/valid/img_top/all_img 
!cp /content/drive/My\ Drive/zpo_project/data/valid/masks_top/*.png /content/valid/mask_top/all_mask

mkdir: cannot create directory ‘/content/train’: File exists
mkdir: cannot create directory ‘/content/valid’: File exists
mkdir: cannot create directory ‘/content/test’: File exists
mkdir: cannot create directory ‘/content/train/img_front’: File exists
mkdir: cannot create directory ‘/content/train/mask_front’: File exists
mkdir: cannot create directory ‘/content/train/img_front/all_img’: File exists
mkdir: cannot create directory ‘/content/train/mask_front/all_mask’: File exists
mkdir: cannot create directory ‘/content/train/img_side’: File exists
mkdir: cannot create directory ‘/content/train/mask_side’: File exists
mkdir: cannot create directory ‘/content/train/img_side/all_img’: File exists
mkdir: cannot create directory ‘/content/train/mask_side/all_mask’: File exists
mkdir: cannot create directory ‘/content/train/img_top’: File exists
mkdir: cannot create directory ‘/content/train/mask_top’: File exists
mkdir: cannot create directory ‘/content/train/img_top/all_img’: File exists


In [ ]:
# variables for traning 

imgs_train = ['/content/train/img_front', '/content/train/img_top', '/content/train/img_side']
masks_train = ['/content/train/mask_front', '/content/train/mask_top', '/content/train/mask_side']

imgs_valid = ['/content/valid/img_front', '/content/valid/img_top', '/content/valid/img_side']
masks_valid = ['/content/valid/mask_front', '/content/valid/mask_top',  '/content/valid/mask_side']
model_paths = ['/content/drive/My Drive/zpo_project/models/side.h5',
               '/content/drive/My Drive/zpo_project/models/front.h5', 
               '/content/drive/My Drive/zpo_project/models/top.h5']

target_size = padding_shape
train_batch_size = 8
valid_batch_size = 16
random_seed = 42


Generators and traning

In [ ]:
for i in range(len(model_paths)):

  # generate generators
  img_gen_train = tf.keras.preprocessing.image.ImageDataGenerator()
  masks_gen_train = tf.keras.preprocessing.image.ImageDataGenerator()

  img_gen_val = tf.keras.preprocessing.image.ImageDataGenerator()

  masks_gen_val = tf.keras.preprocessing.image.ImageDataGenerator()

  train_images_generator = img_gen_train.flow_from_directory(
      imgs_train[i],
      target_size=target_size,
      batch_size=train_batch_size,
      class_mode=None,
      seed=random_seed,
      color_mode="grayscale"
  )

  train_masks_generator = masks_gen_train.flow_from_directory(
      masks_train[i],
      target_size=target_size,
      batch_size=train_batch_size,
      class_mode=None,
      seed=random_seed,
      color_mode="grayscale"
  )

  val_images_generator = img_gen_val.flow_from_directory(
      imgs_valid[i],
      target_size=target_size,
      batch_size=valid_batch_size,
      class_mode=None,
      seed=random_seed,
      color_mode="grayscale"
  )

  val_masks_generator = masks_gen_val.flow_from_directory(
      masks_valid[i],
        target_size=target_size,
      batch_size=valid_batch_size,
      class_mode=None,
      seed=random_seed,
      color_mode="grayscale"
  )

  train_combined_generator = zip(train_images_generator, train_masks_generator)
  val_combined_generator = zip(val_images_generator, val_masks_generator)

  # load model and train it
  training_samples = train_images_generator.n
  validation_samples = val_images_generator.n
  opt = keras.optimizers.Adam(lr=1e-3)
  metric = IOUScore()
  model = Unet('inceptionv3', input_shape=input_shape, encoder_weights=None)
  model.compile(optimizer=opt, loss=loss,
                    metrics=[metric])

  history = model.fit_generator(
    train_combined_generator,
    steps_per_epoch=training_samples // 8,
    epochs=20,
    validation_data=val_combined_generator, 
    validation_steps=validation_samples // 16
  )

  model.save(model_paths[i])


Found 380 images belonging to 1 classes.
Found 380 images belonging to 1 classes.
Found 111 images belonging to 1 classes.
Found 111 images belonging to 1 classes.
Epoch 1/1
47/47 [==============================] - 152s 3s/step - loss: 0.5753 - iou_score: 0.2891 - val_loss: 0.9848 - val_iou_score: 0.0044
Found 380 images belonging to 1 classes.
Found 380 images belonging to 1 classes.
Found 111 images belonging to 1 classes.
Found 111 images belonging to 1 classes.
Epoch 1/1
47/47 [==============================] - 131s 3s/step - loss: 0.5572 - iou_score: 0.3043 - val_loss: 0.8271 - val_iou_score: 0.0922
Found 380 images belonging to 1 classes.
Found 380 images belonging to 1 classes.
Found 111 images belonging to 1 classes.
Found 111 images belonging to 1 classes.
Epoch 1/1
47/47 [==============================] - 131s 3s/step - loss: 0.6932 - iou_score: 0.1871 - val_loss: 0.8315 - val_iou_score: 0.0954


Make predictions and save it

In [ ]:
# prepare variables


models_paths = ['/content/drive/My Drive/zpo_project/models/side.h5',
               '/content/drive/My Drive/zpo_project/models/front.h5', 
               '/content/drive/My Drive/zpo_project/models/top.h5']

result_1_path = Path('/content/drive/My Drive/zpo_project/results/result_1')
result_2_path = Path('/content/drive/My Drive/zpo_project/results/result_2')

models = []
for model in models_paths:  
  models.append(load_model(model, custom_objects={'dice_loss': loss, 'iou_score': metric}))

In [ ]:
# first data set prediction

test_data_path = Path('/content/FirstDataset/test')

for scan_path in test_data_path.iterdir():

  data, affine = load_raw_volume(scan_path)
  labels = np.zeros(data.shape, dtype=np.float32)

  for i in range(data.shape[0]):
    labels[i] += get_prediction(data[i], models[0], padding_shape)
  for i in range(data.shape[1]):
    labels[:,i] += get_prediction(data[:,i], models[1], padding_shape)
  for i in range(data.shape[2]):
    labels[:,:,i] += get_prediction(data[:,:,i], models[2], padding_shape)

  labels = labels/3
  labels[labels > 0.7] = 1
  labels[labels != 1] = 0

  save_labels(labels, affine, os.path.join(result_1_path, scan_path.name))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
# second data set prediction

test_data_path = Path('/content/SecondDataset/test')

for scan_path in test_data_path.iterdir():

  data, affine = load_raw_volume(os.path.join(scan_path,'T1w.nii.gz'))
  labels = np.zeros(data.shape, dtype=np.float32)

  for i in range(data.shape[0]):
    labels[i] += get_prediction(data[i], models[0], padding_shape)
  for i in range(data.shape[1]):
    labels[:,i] += get_prediction(data[:,i], models[1], padding_shape)
  for i in range(data.shape[2]):
    labels[:,:,i] += get_prediction(data[:,:,i], models[2], padding_shape)


  labels = labels/3
  labels[labels > 0.7] = 1
  labels[labels != 1] = 0

  save_labels(labels, affine, os.path.join(result_2_path, scan_path.name+'.nii.gz'))
  break


Checking results

In [ ]:
import requests
import zlib

for dataset_predictions_path in (result_1_path, result_2_path):
  for prediction_path in dataset_predictions_path.iterdir():
    prediction_name = prediction_path.name[:-7]  # Usuwanie '.nii.gz' z nazwy pliku
    prediction = nib.load(str(prediction_path))

    response = requests.post(f'http://vision.dpieczynski.pl:8080/{prediction_name}', data=zlib.compress(prediction.to_bytes()))
    if response.status_code == 200:
        print(dataset_predictions_path.name, prediction_path.name, response.json())
    else:
        print(f'Error processing prediction {dataset_predictions_path.name}/{prediction_name}: {response.text}')